In [9]:
'''
!!NOTE!!
!!User Input required!!

Parameters:

execution_mode (string): where the notebook is being run
    Sample: 'local', 'in-cluster'

host (string): KF Pipelines service endpoint
    Sample:  "http://10.10.10.10:31380/pipeline"


role_arn (string): SageMaker Role ARN for execution of pipeline components
    Sample: 'arn:aws:iam::${account_id}:role/service-role/AmazonSageMaker-ExecutionRole-${timestemp}'
'''

execution_mode = "local"
host = "" 
role_arn = ''

if execution_mode == "local" and host == None:
    raise ValueError("Please set host to the appropriate URL")
elif execution_mode != "local":
    execution_mode = "in-cluster"

In [10]:
import kfp
import kfp.dsl as dsl
from kfp import components
from kubernetes import client as k8s_client
from kfp.aws import use_aws_secret
import logging

In [11]:
# Set loglevel to debug
logging.basicConfig()
logging.getLogger().setLevel(logging.DEBUG)

model = 'https://raw.githubusercontent.com/kubeflow/pipelines/master/components/aws/sagemaker/model/component.yaml'
deploy = 'https://raw.githubusercontent.com/kubeflow/pipelines/master/components/aws/sagemaker/deploy/component.yaml'

In [12]:
sagemaker_model_op = components.load_component_from_url(model)
sagemaker_deploy_op = components.load_component_from_url(deploy)

def blerssi_mxnet_train_upload_op(step_name='mxnet-train'):
    return dsl.ContainerOp(
        name='mxnet-train-upload-s3',
        image='ciscoai/mxnet-blerssi-train-upload:v0.1.1',
    ).apply(use_aws_secret(secret_name='aws-secret', aws_access_key_id_name='AWS_ACCESS_KEY_ID', aws_secret_access_key_name='AWS_SECRET_ACCESS_KEY'))

In [13]:
@dsl.pipeline(
    name='MXNet Sagemaker Hybrid Pipeline',
    description='Pipeline to train BLERSSI model using mxnet and save in aws s3 bucket'
)
def mxnet_pipeline(
    region="",
    image="",
    model_name="",
    endpoint_config_name="",
    endpoint_name="",
    model_artifact_url="",
    instance_type_1="",
    role=""
):
    train_upload_model = blerssi_mxnet_train_upload_op()

    create_model = sagemaker_model_op(
        region=region,
        model_name=model_name,
        image=image,
        model_artifact_url=model_artifact_url,
        role=role
    ).apply(use_aws_secret(secret_name='aws-secret', aws_access_key_id_name='AWS_ACCESS_KEY_ID', aws_secret_access_key_name='AWS_SECRET_ACCESS_KEY'))
    create_model.after(train_upload_model)

    sagemaker_deploy_op(
        region=region,
        endpoint_config_name=endpoint_config_name,
        endpoint_name=endpoint_name,
        model_name_1=create_model.output,
        instance_type_1=instance_type_1
    ).apply(use_aws_secret(secret_name='aws-secret', aws_access_key_id_name='AWS_ACCESS_KEY_ID', aws_secret_access_key_name='AWS_SECRET_ACCESS_KEY'))

In [14]:
try:
    import kfp.compiler as compiler
    compiler.Compiler().compile(mxnet_pipeline, 'mxnet_pipeline.tar.gz')
except RuntimeError as err:
    logging.debug(err)
    logging.info("Argo workflow failed validation check but it can still be used to run experiments.")

DEBUG:root:Internal compiler error: Compiler has produced Argo-incompatible workflow.
Please create a new issue at https://github.com/kubeflow/pipelines/issues attaching the pipeline code and the pipeline package.
Error: time="2020-05-11T16:48:48-07:00" level=fatal msg="/dev/stdin: spec.arguments.parameters[2].name: 'model_name' is invalid: name must consist of alpha-numeric characters or '-', and must start with an alpha-numeric character (e.g. My-name1-2, 123-NAME)"

INFO:root:Argo workflow failed validation check but it can still be used to run experiments.


In [15]:
client = None
if execution_mode == "local":
    client = kfp.Client(host=host)
else:
    client = kfp.Client()
blerssi_hybrid_experiment = client.create_experiment(name='BLERSSI-Sagemaker')

In [17]:
from datetime import datetime
timestamp = datetime.now().strftime("%d-%m-%y-%H-%M-%S")
logging.info("timestamp for pipeline artifacts: %s", timestamp)

# pipeline parameters :

# Region where the pipeline is supposed to push/pull artifacts
aws_region = 'us-west-2'

# pre-built inference image for serving the mxnet BLERSSI model
inference_image = '609615689023.dkr.ecr.us-west-2.amazonaws.com/mxnet-blerssi-inference:latest'

# model name to create a re-usable SageMaker Model resource
model_name = 'mxnet-blerssi-'+timestamp

# endpoint config name for the SageMaker Model Serving Endpoint Config
endpoint_config_name = 'mxnet-blerssi-endpoint-config-'+timestamp

# endpoint name for SageMaker Serving Endpoint
endpoint_name = 'mxnet-blerssi-'+timestamp

# model artifact URL
# Path to the model tarball 
model_path = 's3://mxnet-model-store/blerssi/model.tar.gz'

# AWS instance type
instance_type = 'ml.m4.xlarge'

run = client.run_pipeline(blerssi_hybrid_experiment.id, 'blerssi-sagemaker-pipeline-'+timestamp, pipeline_package_path='mxnet_pipeline.tar.gz', params={
    'region': aws_region,
    'image': inference_image,
    'model_name': model_name,
    'endpoint_config_name': endpoint_config_name,
    'endpoint_name': endpoint_name,
    'model_artifact_url': model_path,
    'instance_type_1': instance_type,
    'role': role_arn
})